<a href="https://colab.research.google.com/github/insjang-muhayu/note-python-ai/blob/main/%5B%EA%B8%88%EC%9C%B5AI%5D%EA%B8%B0%EB%A7%90%EA%B3%BC%EC%A0%9C_%EC%9E%A5%EC%9D%B8%EC%88%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기말과제 - Financial AI Final
---
* 강좌명 : 금융 AI
* 제출자 : 장인순 (insjang)
* 이메일 : insjang@naver.com
---

## Part1. 기말과제 - 기본 수집 가공 소스

### 1.1. 패키지 설치 및 라이브러리 가져오기

In [ ]:
!pip install -U finance-datareader
!pip install -U pandas_datareader

Requirement already up-to-date: pandas_datareader in /usr/local/lib/python3.7/dist-packages (0.9.0)


In [ ]:
import os, csv, time, math

import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timedelta

import pandas_datareader.data as pdr
import FinanceDataReader as fdr

import statsmodels.api as sm
from scipy.stats import skew, kurtosis

import matplotlib.pyplot as plt
import seaborn as sns

from concurrent import futures

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 10)

plt.style.use('seaborn-whitegrid') #('fivethirtyeight')
%matplotlib inline


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Default Date : 2010.01.04 ~ 2020.11.30
BASE_SDATE = dt.datetime(2010, 1, 4)
BASE_EDATE = dt.datetime(2020,11,30)

KRX_LIST = None
TERM_LST = ['m1', 'm3', 'm6', 'm9', 'y1']
DAYS_LST = [23, 65, 130, 190, 253]

ROOT_PATH = '/content/drive/MyDrive/data/stock'
DATA_PATH = ROOT_PATH+'/output'

def dfcsv_read(filenm) :
    return pd.read_csv(f"{DATA_PATH}/{filenm}", index_col='Date', parse_dates=True)

def dfcsv_save(df, filenm) :
    df.to_csv(f"{DATA_PATH}/{filenm}", quoting=csv.QUOTE_MINIMAL)

os.listdir(ROOT_PATH)

['krx_data', 'output', '.ipynb_checkpoints']

### 1.2. 데이터 수집 (KOSPI)

#### 코스피 종목 수집 (krx)

In [ ]:
def make_krx() : # 한국거래소에 있는 주식의 데이터를 가져옴
    df_krx = None
    if not os.path.exists(f"{DATA_PATH}/krx.csv") :
        df_krx = fdr.StockListing('KOSPI')
        df_krx.set_index('Symbol', inplace=True)
        dfcsv_save(df_krx, 'krx.csv')
    else : 
        df_krx = pd.read_csv(f"{DATA_PATH}/krx.csv", dtype=str, index_col='Symbol')

    return df_krx

krx = make_krx()
KRX_LIST = krx.index.tolist()

#### 주식종가(stock_cls), 주식수익률(stock_rtn) 가공

In [ ]:
def download(code, sday=BASE_SDATE, eday=BASE_EDATE):
    df = fdr.DataReader(code, sday, eday)
    dfnew = df.copy(); dfnew['Symbol'] = code
    dfnew.to_csv(f"{ROOT_PATH}/krx_data/{code}.csv", quoting=csv.QUOTE_MINIMAL)

def collect_stock() :
    if not os.path.exists(ROOT_PATH+'/krx_data') :
        os.mkdirs(ROOT_PATH+'/krx_data')
        workers = min(30, len(KRX_LIST))
        with futures.ThreadPoolExecutor(workers) as executor:
            res = executor.map(download, KRX_LIST)

def make_stock() :
    global KRX_LIST
    df_cls = None; df_rtn = None
    if not os.path.exists(f"{DATA_PATH}/stock_cls.csv") :
        df = pd.DataFrame(); del_lst = []
        for item in KRX_LIST:
            if not os.path.isfile(f'{ROOT_PATH}/krx_data/{item}.csv'): continue
            tmp = pd.read_csv(f'{ROOT_PATH}/krx_data/{item}.csv', parse_dates=True)
            if ('Close' in tmp.columns) and ('Date' in tmp.columns):
                tmp.set_index('Date', inplace=True, drop=False)
                tmp_close = tmp['Close'].rename(item)
                df = pd.concat([df, tmp_close], axis=1, sort=True)
            else : del_lst.append(item)

        for item in del_lst : 
            KRX_LIST.remove(item)
            os.remove(f'{ROOT_PATH}/krx_data/{item}.csv')

        df_cls = df.loc[:, ~df.columns.duplicated()]; # 주식종가
        with np.errstate(divide='ignore'): # 주식수익률
            df_rtn = np.round(100. * np.log((df_cls / df_cls.shift(1)).astype('float')), 2)

        dfcsv_save(df_cls, 'stock_cls.csv')
        dfcsv_save(df_rtn, 'stock_rtn.csv')
    else :
        df_cls = dfcsv_read('stock_cls.csv')
        df_rtn = dfcsv_read('stock_rtn.csv')

    return df_cls, df_rtn

# 주식별 주가 수집
collect_stock()

stock_cls, stock_rtn = make_stock()

#### 지수종가(kospi_cls), 지수수익률(kospi_rtn) 가공

In [ ]:
def make_kospi(sday=BASE_SDATE, eday=BASE_EDATE) :
    df = None; df_cls = None; df_rtn = None
    if not os.path.exists(f"{DATA_PATH}/kospi.csv") :
        df = pdr.DataReader('^KS11', 'yahoo', sday, eday) #KOSPI
        df.index = pd.to_datetime(df.index).strftime("%Y-%m-%d")
        dfcsv_save(df, 'kospi.csv')
    else : df = dfcsv_read('kospi.csv')

    df_cls = (np.round(df['Close'], 2)).rename('kospi_close') # 코스피 close
    df_rtn = (np.round(100. * np.log(df_cls / df_cls.shift(1)), 2)).rename('kospi_return')
    
    dfcsv_save(df_cls, 'kospi_cls.csv')
    dfcsv_save(df_rtn, 'kospi_rtn.csv')

    return df, df_cls, df_rtn

kospi, kospi_cls, kospi_rtn = make_kospi()

### 1.3. 데이터 가공 (모멘텀, 포트폴리오)

#### 모멘텀 구성 (mmt_m1, mmt_m3, mmt_m6, mmt_m9, mmt_y1)
* M1:23일 / M3:65일 / M6:130일 / M9:190일 / Y1:253일

In [ ]:
def getMoment(df, term, days) :
    df_mmt = None
    if not os.path.exists(f"{DATA_PATH}/mmt_{term}.csv") :
        lagg = df.shift(1); cols = df.columns
        df_mmt = round(lagg.pct_change(periods=days)*100, 2)
        df_mmt.columns = [f'{x}' for x in cols]
        dfcsv_save(df_mmt, f'mmt_{term}.csv'); print('[신규생성]', f'mmt_{term}.csv')
    else : df_mmt = dfcsv_read(f'mmt_{term}.csv')
    return df_mmt

MOMT_LST = []
for idx in range(len(TERM_LST)) :
    MOMT_LST.append(getMoment(stock_cls, TERM_LST[idx], DAYS_LST[idx]))

MOMT_LST[0]

,095570,006840,152100,295820,253150,253160,278420,292750,309170,309210,333940,333950,333960,333970,333980,269530,251590,161510,251600,289670,298340,189400,278620,269540,287180,213630,332610,332620,195970,238670,195980,373530,256450,239660,280920,266550,301400,301410,227830,122090,328370,301440,376250,027410,282330,138930,001460,001465,001040,079160,...,011760,004310,322000,017800,307950,011210,267260,004020,267250,005380,005387,005389,005385,001500,227840,126560,001450,057050,093240,003010,111110,008770,008775,002460,013520,241590,006060,010690,133820,010660,000850,016580,032560,004800,094280,298040,298050,298020,298000,093370,081660,005870,079980,005010,069260,215620,000540,000547,000545,003280
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-24,23.18,19.21,10.22,8.58,21.19,-18.88,4.35,10.21,12.14,10.86,8.13,10.63,8.87,10.99,7.78,5.10,6.87,9.61,3.97,-0.40,-0.05,4.82,0.04,3.13,1.31,7.61,-1.81,2.12,7.62,1.63,7.32,NaN,-0.13,0.04,9.80,7.55,4.93,-5.98,10.21,11.00,9.81,9.41,NaN,11.67,-3.31,7.13,22.12,13.22,-1.23,1.40,...,5.57,1.73,9.08,1.75,23.73,13.11,20.61,6.08,18.18,8.16,5.09,7.36,8.48,19.52,3.18,-1.36,1.67,-1.99,50.77,-1.82,9.18,5.18,21.20,-3.12,3.67,0.34,-3.81,3.70,15.07,3.31,2.57,6.80,1.41,0.39,-3.47,0.16,7.04,26.01,4.49,3.76,11.22,3.85,-2.88,4.43,5.36,4.12,-2.81,-8.91,-28.71,0.0
2020-11-25,23.57,24.23,11.59,8.16,24.36,-20.74,4.66,10.59,14.99,11.90,8.26,11.55,9.73,10.75,7.60,5.11,6.08,9.02,3.19,-0.67,-0.07,5.35,0.02,5.24,2.62,9

#### 포트폴리오 구성 (pfo_m1, pfo_m3, pfo_m6, pfo_m9, pfo_y1)

In [ ]:
# 포트폴리오
pfo_m1 = None; pfo_m3 = None; pfo_m6 = None; pfo_m9 = None; pfo_y1 = None;

def getPortfo(df, term, sday, eday) :
    if not os.path.exists(f"{DATA_PATH}/pfo_{term}.csv") :
        cols = df.columns
        top = []; mid = []; bot = []
        df_st = pd.DataFrame(columns=[f'sT{x}' for x in range(10)]);
        df_sm = pd.DataFrame(columns=[f'sM{x}' for x in range(10)]);
        df_sb = pd.DataFrame(columns=[f'sB{x}' for x in range(10)]);
        df_mt = pd.DataFrame(columns=[f'mT{x}' for x in range(10)]);
        df_mm = pd.DataFrame(columns=[f'mM{x}' for x in range(10)]);
        df_mb = pd.DataFrame(columns=[f'mB{x}' for x in range(10)]);

        for day in df.loc[sday:eday].index:
            row = df.loc[day].transpose().dropna()
            row = row.sort_values(ascending=False)
            sz = row.size; mi = int(sz/2)
            if sz < 150 : continue

            idx = pd.to_datetime(day, format='%Y-%m-%d')
            df_st.loc[idx] = ['#'+x for x in row.iloc[20:30].index.tolist()]
            df_sm.loc[idx] = ['#'+x for x in row.iloc[mi-5:mi+5].index.tolist()]
            df_sb.loc[idx] = ['#'+x for x in row.iloc[sz-60:sz-50].index.tolist()]
            df_mt.loc[idx] = [round(x, 2) for x in row.iloc[20:30].tolist()]
            df_mm.loc[idx] = [round(x, 2) for x in row.iloc[mi-5:mi+5].tolist()]
            df_mb.loc[idx] = [round(x, 2) for x in row.iloc[sz-60:sz-50].tolist()]

        df_pfo = df_st.join(df_sm).join(df_sb).join(df_mt).join(df_mm).join(df_mb).dropna();
        df_pfo.index.name = 'Date'

        dfcsv_save(df_pfo, f'pfo_{term}.csv'); print('[신규생성]', f'pfo_{term}.csv')
    else :
        df_pfo = dfcsv_read(f'pfo_{term}.csv')

    return df_pfo

POFO_LST = []
for idx in range(len(TERM_LST)) :
    POFO_LST.append(getPortfo(MOMT_LST[idx], TERM_LST[idx], '2012', '2014'))

POFO_LST[0]

,sT0,sT1,sT2,sT3,sT4,sT5,sT6,sT7,sT8,sT9,sM0,sM1,sM2,sM3,sM4,sM5,sM6,sM7,sM8,sM9,sB0,sB1,sB2,sB3,sB4,sB5,sB6,sB7,sB8,sB9,mT0,mT1,mT2,mT3,mT4,mT5,mT6,mT7,mT8,mT9,mM0,mM1,mM2,mM3,mM4,mM5,mM6,mM7,mM8,mM9,mB0,mB1,mB2,mB3,mB4,mB5,mB6,mB7,mB8,mB9
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2012-01-02,#002700,#003230,#002880,#000040,#003560,#030720,#000890,#051630,#005740,#037270,#014680,#137930,#002310,#140580,#102110,#015360,#122630,#108590,#006200,#139310,#013520,#007660,#000850,#009240,#023590,#019440,#010130,#021050,#011000,#009160,43.16,42.78,42.78,41.48,40.77,40.20,38.26,38.21,37.92,36.25,1.90,1.89,1.87,1.83,1.82,1.80,1.78,1.78,1.77,1.74,-10.03,-10.10,-10.30,-10.35,-10.43,-10.44,-10.46,-10.48,-10.49,-10.53
2012-01-03,#033180,#004840,#023960,#002700,#008600,#118000,#005690,#015260,#001520,#007570,#009770,#130730,#145850,#137930,#094800,#003555,#000660,#101280,#140710,#090350,#005320,#006280,#001230,#012510,#004430,#002350,#001440,#002020,#122900,#007660,44.99,44.69,44.51,44.30,43.05,42.12,40.50,39.77,38.41,38.26,0.28,0.27,0.26,0.26,0.24,0.22,0.21,0.20,0.20,0.17,-10.76,-11.24,-11.38,-11.48,-11.48,-11.52,-11.55,-11.94,-12.30,-12.33
2012-01-04,#011690,#003560,#002700,#025750,#005690,#006980,#005190,#007570,#016450,#000890,#000155,#024720,#004020,#000725,#094280,#004870,#000815,#010770,#003830,#107590,#139320,#003570,#020560,#011090,#017550,#009190,#023590,#096775,#013520,#000080,46.21,44.56,42.53,41.78,40.85,40.40,40.22,38.07,38.02,36.36,1.51,1.46,1.46,1.35,1.33,1.31,1.22,1.20,1.20,1.20,-8.99,-9.09,-9.22,-9.35,-9.55,-9.67,-9.70,-9.77,-10.09,-10.17
2012-01-05,#000180,#033180,#003560,#025750,#023960,#007570,#003230,#002700,#002880,#051630,#090350,#004985,#101140,#011070,#140710,#008930,#128820,#090435,#009410,#138520,#005950,#006650,#079430,#020560,#144600,#011810,#007460,#051915,#122900,#011785,40.77,40.73,40.53,40.53,39.19,38.48,36.61,36.53,35.70,35.66,0.00,0.00,0.00,0.00,0.00,-0.03,-0.18,-0.18,-0.18,-0.21,-11.15,-11.41,-11.51,-11.54,-11.78,-11.83,-11.90,-12.03,-12.08,-12.30
2012-01-06,#023150,#000180,#002700,#033180,#000890,#008600,#004840,#007810,#023960,#002880,#058430,#000060,#131890,#006805,#066575,#004990,#009070,#019680,#090435,#006740,#001230,#018500,#011810,#011790,#000885,#078930,#007860,#031430,#010660,#020560,40.17,40.14,40.00,39.46,38.50,38.00,37.76,37.29,36.89,36.78,0.00,0.00,-0.10,-0.11,-0.21,-0.29,-0.32,-0.32,-0.36,-0.36,-11.36,-11.59,-11.65,-11.97,-12.07,-12.09,-12.16,-12.17,-12.18,-12.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-23,#006490,#005430,#010580,#007810,#010950,#033240,#090350,#000390,#004980,#004710,#101280,#000140,#015760,#104530,#010040,#007980,#015360,#009140,#005720,#017670,#138490,#020150,#000215,#111770,#001360,#012610,#030790,#051600,#144620,#006360,23.40,23.36,22.71,22.21,22.16,21.60,20.75,19.82,19.79,19.56,-1.96,-1.97,-1.98,-1.99,-2.00,-2.02,-2.04,-2.06,-2.06,-2.07,-14.40,-14.51,-14.53,-14.67,-14.94,-14.96,-15.03,-15.28,-15.50,-15.86
2014-12-24,#081000,#000390,#004980,#118000,#001780,#079430,#007660,#008930,#004710,#090350,#144600,#104530,#001420,#029530,#051630,#205720,#000650,#068270,#002025,#000150,#021050,#010600,#017800,#009270,#001530,#001740,#020150,#067830,#001430,#009310,23.55,23.53,23.29,22.00,20.86,20.62,19.05,18.78,18.72,18.64,-1.60,-1.64,-1.68,-1.69,-1.69,-1.71,-1.74,-1.74,-1.75,-1.76,-13.50,-13.57,-13.76,-13.76,-13.81,-14.27,-14.33,-14.35,-14.37,-14.49
2014-12-26,#004980,#006490,#090350,#00781K,#004985,#192090,#168580,#204450,#003410,#010950,#008040,#105560,#023800,#101280,#104530,#192720,#086280,#200250,#001755,#014530,#138490,#001440,#021050,#035720,#009970,#000500,#161000,#033250,#051600,#017800,23.71,22.46,22.31,22.01,21.86,21.75,21.61,21.27,21.16,20.81,-1.50,-1.52,-1.53,-1.54,-1.56,-1.57,-1.58,-1.60,-1.63,-1.65,-13.36,-13.37,-13.39,-13.41,-13.48,-1

## Part2. 기말과제 실습 수행

In [ ]:
krx = pd.read_csv(f"{DATA_PATH}/krx.csv", dtype=str, index_col=0)
stock_cls = dfcsv_read('stock_cls.csv')
stock_rtn = dfcsv_read('stock_rtn.csv')

kospi = dfcsv_read('kospi.csv')
kospi_cls = pd.read_csv(f"{DATA_PATH}/kospi_cls.csv", index_col=0, parse_dates=True, header=0, squeeze=True)
kospi_rtn = pd.read_csv(f"{DATA_PATH}/kospi_rtn.csv", index_col=0, parse_dates=True, header=0, squeeze=True)

### 문제1. KOSPI지수와 NAVER주식의 종가 및 수익률 결과 출력

In [ ]:
# NAVER(035420) / KOSPI
df = pd.DataFrame({
    'nvr_cls' : stock_cls['035420'],
    'nvr_rtn' : stock_rtn['035420'],
    'ksp_cls' : kospi_cls,
    'ksp_rtn' : kospi_rtn
}).dropna()
df

### 문제2. 모멘텀 계산 (1M:23, 3M:65, 6M:130, 9M:190, 1Y:253)

In [ ]:
df['mmt_1m'] = 100. * np.log(df.nvr_cls / df.nvr_cls.shift(23))
df['mmt_3m'] = 100. * np.log(df.nvr_cls / df.nvr_cls.shift(65))
df['mmt_6m'] = 100. * np.log(df.nvr_cls / df.nvr_cls.shift(130))
df['mmt_9m'] = 100. * np.log(df.nvr_cls / df.nvr_cls.shift(190))
df['mmt_1y'] = 100. * np.log(df.nvr_cls / df.nvr_cls.shift(253))
df.index.name = 'Date'
df

### 문제3. 포트폴리오 구성 - best(21-30), median(10), worst(51-60)

In [ ]:
MOMT_LST = []; POFO_LST = []
for idx in range(len(TERM_LST)) :
    MOMT_LST.append(dfcsv_read(f'mmt_{TERM_LST[idx]}.csv')) # 모멘텀
    POFO_LST.append(dfcsv_read(f'pfo_{TERM_LST[idx]}.csv')) # 포트폴리오

POFO_LST[0]

#### (a) 주식 모멘텀 일별 출력

In [ ]:
# term - m1, m3, m6, m9, y1
# opts - m:모멘텀, s:심볼
def get_fields(term, opts) :
    colnms = []
    for gdx in ['T', 'M', 'B'] : # best, median, worst
        for rdx in range(10) : # rank : 0 ~ 9
            colnms.append(f'{opts}{gdx}{rdx}')
    return colnms

for idx in range(len(TERM_LST)) :
    pfo = POFO_LST[idx]
    print(pfo[ get_fields(TERM_LST[idx], 'm') ], '\n')

#### (b) 주식 회사명 일별 출력

In [ ]:
codes = {}
for idx in krx.index.tolist() :
    key = '#'+idx; val = krx.loc[idx, 'Name']
    codes[key] = val; #print(f'{key}\t:[{val}]')

def getPortfoTicker(df, prefix) :
    df_res = df.copy(); cols = get_fields(prefix, 's')
    for idx in cols : df_res[idx] = df[idx].apply(lambda x: codes[x])
    return df_res[cols]

# def getPortfoTicker(df, term) :
#     df_res = df.copy(); cols = get_fields(term, 's')    
#     src = ['#'+x for x in krx.index.tolist()]
#     tgt = krx.Name.tolist()
#     return df_res[cols].replace(src, tgt)

PTCK_LST = []
for idx in range(len(TERM_LST)) :
    pfo = POFO_LST[idx]
    tck = getPortfoTicker(pfo, TERM_LST[idx])
    PTCK_LST.append(tck); print(tck, '\n')


#### (c) 1일 보유수익률 출력

In [ ]:
# print(stock_rtn.loc['2010-01-05', '006840'])
# POFO_LST[0]

def getPortfoReturn(df, term) :
    cols = get_fields(term, 's')
    df_res = df[cols]
    for day in df.index.tolist() :
        for col in cols :
            x = df.loc[day, col][1:] # 첫문자 절삭 (#006840)
            df_res.loc[day, col] = stock_rtn.loc[day, x]
    return df_res

PRTN_LST = []
for idx in range(len(TERM_LST)) :
    pfo = POFO_LST[idx]
    rtn = getPortfoReturn(pfo, TERM_LST[idx])
    PRTN_LST.append(rtn); print(rtn, '\n')


#### (d) 15개 포트폴리오 일별 평균 수익률

In [ ]:
# print(stock_rtn.loc['2010-01-05', '006840'])
# POFO_LST[0]

def getPortfoMean(df, term) :
    df_res = df.copy()
    for level in ['T', 'M', 'B'] :
        df_res[term+'-'+level] = df[[f's{level}{x}' for x in range(10)]].mean(axis=1)

    return df_res[[term+'-'+x for x in ['T', 'M', 'B']]]

PAVG_LST = []
for idx in range(len(TERM_LST)) :
    rtn = PRTN_LST[idx]
    avg = getPortfoMean(rtn, TERM_LST[idx])
    PAVG_LST.append(avg)

#### (e) 15개 포트폴리오 수익률 출력

In [ ]:
pofol_rtn = pd.concat(PAVG_LST, axis=1); 
pofol_rtn.columns = ['1m_best', '1m_medi', '1m_wrst', 
                     '3m_best', '3m_medi', '3m_wrst', 
                     '6m_best', '6m_medi', '6m_wrst', 
                     '9m_best', '9m_medi', '9m_wrst', 
                     '12m_best', '12m_medi', '12m_wrst']
pofol_rtn

### 문제4. 포트폴리오 & KOSPI 수익률의 기초통계

In [ ]:
pfo_ksp = pd.merge(pofol_rtn, kospi_rtn, how='inner', on='Date', left_index=True)
pfo_ksp

In [ ]:
stats = pfo_ksp.describe()
stats.loc['var'] = pfo_ksp.var().tolist()    # 분산(Variance)
stats.loc['skew'] = pfo_ksp.skew().tolist()  # 왜도(Skewness)
stats.loc['kurt'] = pfo_ksp.kurtosis().tolist()   # 첨도(Kurtosis)
print(stats)

In [ ]:
# 기초통계 Heatmap
plt.figure(figsize = (14,10))
ax = sns.heatmap(stats[1:].corr(), annot=True, fmt='.2f', linewidths=.5, cmap='YlGnBu')
plt.title('Portfolio & KOSPI Stats Heatmap', fontsize=20)
plt.show() 

### 문제5. 포트폴리오 & KOSPI 수익률의 상관관계 시각화

In [ ]:
# 상관관계 Heatmap
plt.figure(figsize = (14,10))
ax = sns.heatmap(pfo_ksp.corr(), annot=True, fmt='.2f', linewidths=.5, cmap='YlGnBu')
plt.title('Portfolio & KOSPI Return Heatmap', fontsize=20)
plt.show() 

In [ ]:
sns.pairplot(pfo_ksp)
plt.title('Portfolio & KOSPI Return Pairplot', fontsize=20)
plt.show() 

### 문제6. 포트폴리오 & KOSPI 수익률의 누적 수익률

In [ ]:

cols = ['1m_best', '1m_medi', '1m_wrst', 
        '3m_best', '3m_medi', '3m_wrst', 
        '6m_best', '6m_medi', '6m_wrst', 
        '9m_best', '9m_medi', '9m_wrst', 
        '12m_best', '12m_medi', '12m_wrst',
        'kospi_return']

cum_rtn = {}
for itm in cols :
    cum_rtn[itm] = pfo_ksp[itm].cumsum()

pfo_ksp_cum = pd.DataFrame(cum_rtn).dropna()
pfo_ksp_cum

In [ ]:
plt.rcParams['legend.fontsize'] = 16
# mystyle = ['b-', 'b-.', 'b:', 'g-', 'g-.', 'g:', 'r-', 'r-.', 'r:', 'c-', 'c-.', 'c:', 'm-', 'm-.', 'm:', 'k-']
pfo_ksp_cum[['1m_best', '3m_best', '6m_best', '9m_best', '12m_best', 'kospi_return']]\
            .plot(figsize=(14,10), fontsize=12, linewidth=2.0)
plt.title('Best vs. KOSPI', fontsize=20)
plt.show()

In [ ]:
plt.rcParams['legend.fontsize'] = 16
# mystyle = ['b-', 'b-.', 'b:', 'g-', 'g-.', 'g:', 'r-', 'r-.', 'r:', 'c-', 'c-.', 'c:', 'm-', 'm-.', 'm:', 'k-']
pfo_ksp_cum[['1m_medi', '3m_medi', '6m_medi', '9m_medi', '12m_medi', 'kospi_return']]\
            .plot(figsize=(14,10), fontsize=12, linewidth=2.0)
plt.title('Median vs. KOSPI', fontsize=20)
plt.show()

In [ ]:
plt.rcParams['legend.fontsize'] = 16
# mystyle = ['b-', 'b-.', 'b:', 'g-', 'g-.', 'g:', 'r-', 'r-.', 'r:', 'c-', 'c-.', 'c:', 'm-', 'm-.', 'm:', 'k-']
pfo_ksp_cum[['1m_wrst', '3m_wrst', '6m_wrst', '9m_wrst', '12m_wrst', 'kospi_return']]\
            .plot(figsize=(14,10), fontsize=12, linewidth=2.0)
plt.title('Worst vs. KOSPI', fontsize=20)
plt.show()